# Bibliotecas e dados

In [ ]:
# Instala as dependências
!pip install -r ../requirements.txt

In [ ]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import IsolationForest
import seaborn as sns

In [ ]:
dados = pd.read_csv('../data/tratado/dados_cvm_tratados.csv')

In [ ]:
# Evitando logs no plot de gráficos
pd.options.mode.chained_assignment = None

# Isolation Forest


Diante do desafio de identificar e prever problemas relacionados à provisão de perdas em Fundos de Investimento em Direitos Creditórios, é imperativo explorar métodos analíticos avançados para fornecer resultados valiosos. Dentre as abordagens consideradas, destaca-se o uso do algoritmo *Isolation Forest*, devido à sua eficácia em identificar padrões de anomalias em conjuntos de dados complexos, como o cenário apresentado.

O *Isolation Forest* é uma técnica de detecção de anomalias que se baseia em uma abordagem não linear para separar observações atípicas das normais em um conjunto de dados. Ele se destaca por sua capacidade de construir rapidamente um conjunto de árvores de decisão, onde os caminhos mais curtos são mais prováveis de serem trilhados por observações anômalas. Essa característica o torna particularmente adequado para detecção de anomalias em problemas complexos e de alta dimensionalidade, como é o caso das transações e movimentações de créditos em FIDCs.

Considerando o problema exposto, em que o objetivo é prever a ocorrência de problemas relacionados à insuficiência ou ausência de provisão para perdas em FIDCs, a aplicação do *Isolation Forest* pode nos proporcionar ideias diferenciadas. Ao analisar os padrões de transações, comportamentos e interações entre os diferentes tipos de créditos adquiridos pelo fundo, o modelo pode destacar transações ou combinações incomuns que indicam um potencial problema de inadimplência futura.

O processo de detecção de anomalias do *Isolation Forest* nos permitirá identificar de maneira eficaz os casos em que a provisão para perdas pode ser inadequada, seja devido a um número anormalmente elevado de transações de alto risco ou a padrões de crédito inconsistentes. Ao identificar esses pontos fora do padrão, estaremos mais próximos de construir um modelo eficiente e preditivo.

In [ ]:
dados.info()

In [ ]:
dados = dados.drop(columns=['Data_Competencia'])

In [ ]:
dados['Carteira_Classificação_encoded'].value_counts()

- Comercial_Total        3
- Multimercado           8
- Financeiro             5
- Servicos_Total          9
- Industrial              6
- Cartao_Credito          2
- Agronegocio             1
- Factoring               7
- Mercado_Imobiliario      4
- Setor_Publico             0
- Acoes_Judiciais           10

In [ ]:
# Lista de carteiras com índices correspondentes à classificação
classificacao_encoding = [
    'SetorPublico',
    'Agronegocio',
    'Cartao',
    'Comercial',
    'Imobiliario',
    'Financeiro',
    'Industrial',
    'Factoring',
    'Multimercado',
    'Servicos',
    'AcoesJudiciais'
]

In [ ]:
# Separando os dados por carteira
dados_classificacao = {
    'Setor Público': dados.loc[dados['Carteira_Classificação_encoded'] == classificacao_encoding.index('SetorPublico')],
    'Agronegócio': dados.loc[dados['Carteira_Classificação_encoded'] == classificacao_encoding.index('Agronegocio')],
    'Cartão': dados.loc[dados['Carteira_Classificação_encoded'] == classificacao_encoding.index('Cartao')],
    'Comercial': dados.loc[dados['Carteira_Classificação_encoded'] == classificacao_encoding.index('Comercial')],
    'Imobiliário': dados.loc[dados['Carteira_Classificação_encoded'] == classificacao_encoding.index('Imobiliario')],
    'Financeiro': dados.loc[dados['Carteira_Classificação_encoded'] == classificacao_encoding.index('Financeiro')],
    'Industrial': dados.loc[dados['Carteira_Classificação_encoded'] == classificacao_encoding.index('Industrial')],
    'Factoring': dados.loc[dados['Carteira_Classificação_encoded'] == classificacao_encoding.index('Factoring')],
    'Multimercado': dados.loc[dados['Carteira_Classificação_encoded'] == classificacao_encoding.index('Multimercado')],
    'Serviços': dados.loc[dados['Carteira_Classificação_encoded'] == classificacao_encoding.index('Servicos')],
    'Ações Judiciais': dados.loc[dados['Carteira_Classificação_encoded'] == classificacao_encoding.index('AcoesJudiciais')]
}

In [ ]:
dados_classificacao['Serviços'].shape

In [ ]:
dados_classificacao['Industrial'].shape

# Construção do modelo

In [ ]:
def detectar_outliers(dados, contaminacao=0.05, random_state=42):
    # Construindo o modelo Isolation Forest
    clf = IsolationForest(n_estimators=100, max_samples='auto', contamination=contaminacao, max_features=1.0, bootstrap=False, n_jobs=-1, random_state=random_state, verbose=0)
    clf.fit(dados)

    # Predição dos outliers
    pred = clf.predict(dados)
    dados.loc[:, 'anomalias'] = pred

    # Filtrando os outliers
    outliers = dados.loc[dados['anomalias'] == -1]

    # Contagem dos outliers para cada valor único em 'ID_Participante'
    top_outliers = outliers['ID_Participante'].value_counts().head(10)

    return top_outliers

# Analisando os resultados

In [ ]:
dados_raw = pd.read_csv('../data/tratado/dados_cvm_tratados.csv')

def pipeline_analise(classificacao, numero_outliers=2):
    top_outliers = detectar_outliers(dados_classificacao[classificacao])
    if len(top_outliers) < numero_outliers:
        numero_outliers = len(top_outliers)
        
    for i in range(numero_outliers):
        df_analise = dados_raw.query(f'ID_Participante == {top_outliers.index[i]}')

        fig, ax = plt.subplots(figsize=(15, 5))
        ax.plot(df_analise['Data_Competencia'], df_analise['Inadimplencia_Total'], label='Inadimplencia_Total')
        ax.plot(df_analise['Data_Competencia'], df_analise['Provisao_Total'], label='Provisao_Total')
        ax.set_title(f'Inadimplencia_Total e Provisao_Total - ID_Participante: {top_outliers.index[i]}')
        ax.legend()
        plt.show()

In [ ]:
for item in dados_classificacao.keys():
    pipeline_analise(item, numero_outliers=3)